In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
img_w, img_h = 28, 28

In [ ]:
fake_input_batch = np.random.uniform(low=0, high=255, size=(32, 28, 28, 1))

In [ ]:
def res_block(input_tensor: tf.Tensor, in_filters: int, out_filters: int, stride=1) -> tf.Tensor:
    # FIXME - may want to apply batch-norm here
    t = tf.nn.relu(input_tensor)
    
    # Downsample for the skip connection
    # FIXME - should this start close to an identity transformation?
    kernel_ds = tf.Variable(tf.truncated_normal(shape=(1, 1, in_filters, out_filters), dtype=tf.float32, stddev=1e-4))
    t_downsampled = tf.nn.conv2d(t, kernel_ds, strides=(1, stride, stride, 1), padding='SAME')
    
    kernel_c1 = tf.Variable(tf.truncated_normal(shape=(3, 3, in_filters, out_filters), dtype=tf.float32, stddev=1e-4))
    t = tf.nn.conv2d(t, kernel_c1, strides=(1, stride, stride, 1), padding='SAME')
    # FIXME - should also apply batch norm here to match Chen et al.
    t = tf.nn.relu(t)
    
    kernel_c2 = tf.Variable(tf.truncated_normal(shape=(3, 3, out_filters, out_filters), dtype=tf.float32, stddev=1e-4))
    t = tf.nn.conv2d(t, kernel_c2, strides=(1, 1, 1, 1), padding='SAME')
    
    # Skip connection
    # N.B. No activation function. This is the sum of the output of two convolutions (one 3x3, the other 1x1).
    return t + t_downsampled


In [ ]:
# Test that our residual block works gives the correct shape of outputs
test_ph1 = tf.placeholder(shape=(None, 28, 28, 1), dtype=tf.float32)
test_out1 = res_block(test_ph1, in_filters=1, out_filters=64, stride=2)
print('Output shape should be [None, 14, 14, 64]: {}'.format(test_out1.shape))

test_ph2 = tf.placeholder(shape=(None, 28, 28, 64), dtype=tf.float32)
test_out2 = res_block(test_ph2, in_filters=64, out_filters=64, stride=1)
print('Output shape should be [None, 28, 28, 64]: {}'.format(test_out2.shape))

# Check that these will actually run
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(test_out1, feed_dict={test_ph1: fake_input_batch})
    sess.run(test_out2, feed_dict={test_ph2: np.tile(fake_input_batch, reps=(1, 1, 1, 64))}) # Tile to fake channels

In [ ]:
def downsample_net(input_tensor: tf.Tensor, n_filters: int) -> tf.Tensor:
    # Convolution with 3x3 kernels
    kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 1, n_filters), stddev=1e-5, dtype=tf.float32))
    biases = tf.Variable(tf.zeros(shape=(1, 1, 1, n_filters), dtype=tf.float32))
    conv_output = tf.nn.conv2d(input_tensor, kernel, strides=1, padding='SAME') + biases
    # FIXME - two downsampling ResBlocks